In [13]:
import pandas as pd 
import datetime as dt

In [4]:
df_ = pd.read_csv("armut_data.csv")

In [5]:
df = df_.copy()
df.head()

,UserId,ServiceId,CategoryId,CreateDate
0,25446,4,5,2017-08-06 16:11:00
1,22948,48,5,2017-08-06 16:12:00
2,10618,0,8,2017-08-06 16:13:00
3,7256,9,4,2017-08-06 16:14:00
4,25446,48,5,2017-08-06 16:16:00


In [8]:
df["Hizmet"] = df["ServiceId"].astype(str) + "_"+ df["CategoryId"].astype(str)

In [10]:
df.head()

,UserId,ServiceId,CategoryId,CreateDate,Hizmet
0,25446,4,5,2017-08-06 16:11:00,4_5
1,22948,48,5,2017-08-06 16:12:00,48_5
2,10618,0,8,2017-08-06 16:13:00,0_8
3,7256,9,4,2017-08-06 16:14:00,9_4
4,25446,48,5,2017-08-06 16:16:00,48_5


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162523 entries, 0 to 162522
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   UserId      162523 non-null  int64 
 1   ServiceId   162523 non-null  int64 
 2   CategoryId  162523 non-null  int64 
 3   CreateDate  162523 non-null  object
 4   Hizmet      162523 non-null  object
dtypes: int64(3), object(2)
memory usage: 6.2+ MB


In [29]:
df["CreateDate"]=pd.to_datetime(df["CreateDate"])

In [32]:
df["NewDate"] = df["CreateDate"].dt.strftime("%Y-%m")

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162523 entries, 0 to 162522
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   UserId      162523 non-null  int64         
 1   ServiceId   162523 non-null  int64         
 2   CategoryId  162523 non-null  int64         
 3   CreateDate  162523 non-null  datetime64[ns]
 4   Hizmet      162523 non-null  object        
 5   NewDate     162523 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 7.4+ MB


In [36]:
df["SepetId"] = df["UserId"].astype(str) + "_" + df["NewDate"]

,Hizmet
SepetId,
0_2017-08,2
0_2017-09,2
0_2018-01,3
0_2018-04,3
10000_2017-08,1
...,...
99_2017-12,4
99_2018-01,2
99_2018-02,1


In [57]:
new_df = df.groupby(["SepetId","Hizmet"])["Hizmet"].count().unstack().fillna(0)\
.applymap(lambda x: 1 if x>0 else 0)

In [58]:
new_df.head()

Hizmet,0_8,10_9,11_11,12_7,13_11,14_7,15_1,16_8,17_5,18_4,...,46_4,47_7,48_5,49_1,4_5,5_11,6_7,7_3,8_5,9_4
SepetId,,,,,,,,,,,,,,,,,,,,,
0_2017-08,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
0_2017-09,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
0_2018-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
0_2018-04,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10000_2017-08,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [59]:
from mlxtend.frequent_patterns import apriori, association_rules

In [60]:
frequent_itemsets = apriori(new_df, min_support=0.01, use_colnames = True)

/Users/huseyinefkanalp/opt/anaconda3/lib/python3.9/site-packages/mlxtend/frequent_patterns/fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [61]:
rules = association_rules(frequent_itemsets, metric = "support", min_threshold=0.01)

In [63]:
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(2_0),(13_11),0.130286,0.056627,0.012819,0.098394,1.737574,0.005442,1.046325
1,(13_11),(2_0),0.056627,0.130286,0.012819,0.226382,1.737574,0.005442,1.124216
2,(15_1),(2_0),0.120963,0.130286,0.033951,0.280673,2.154278,0.018191,1.209066
3,(2_0),(15_1),0.130286,0.120963,0.033951,0.260588,2.154278,0.018191,1.188833
4,(33_4),(15_1),0.027310,0.120963,0.011233,0.411311,3.400299,0.007929,1.493211


In [64]:
def arl_recommender(rules_df, product_id, rec_count = 1):
    sorted_rules = rules_df.sort_values("lift", ascending=False)
    recommendation_list = []
    for i, product in sorted_rules["antecedents"].items():
        for j in list(product):
            if j == product_id:
                recommendation_list.append(list(sorted_rules.iloc[i]["consequents"]))
    recommendation_list = list({item for item_list in recommendation_list for item in item_list})
    return recommendation_list[:rec_count]

In [65]:
arl_recommender(rules, "2_0", 2)

['22_0', '2_0']